In [1]:
import os
import torch

try:
  from google.colab import drive
  drive.mount('/content/drive')

  os.chdir('/content/drive/MyDrive/GitHub/kornia')
except Exception:   # Not run on Google Colab
  import getpass
  if getpass.getuser() == 'vscode':       # Running in container
    os.chdir('/workspaces/kornia/')
  else:                                   # Running in virtual environment
    os.chdir(os.path.join(os.path.expanduser('~'), 'Documents/Git/kornia/'))

  print(os.getcwd())
  import sys
  sys.path.insert(0, os.getcwd())

/home/yhollander/Documents/Git/kornia


In [2]:
from kornia.nerf.camera_utils import parse_colmap_output, create_spline_curve, parse_colmap_points_3d
from kornia.nerf.rays import FocalAxisRay, sample_lengths, sample_ray_points

## Load Colmap outputs

### Scene Colmap camera poses

In [3]:
scene_name = 'head_sculpture4'
data_dir = 'kornia/nerf/kornia_nerf_data/'

scene_dir = os.path.join(data_dir, scene_name)

cameras_path = os.path.join(scene_dir, 'cameras.txt')
images_path = os.path.join(scene_dir, 'images.txt')

In [4]:
img_names, cameras = parse_colmap_output(cameras_path, images_path, 'cpu', torch.float32, sort_by_image_names=True)
print(img_names)


['HeadSculpture_001.jpeg', 'HeadSculpture_002.jpeg', 'HeadSculpture_003.jpeg', 'HeadSculpture_004.jpeg', 'HeadSculpture_005.jpeg', 'HeadSculpture_006.jpeg', 'HeadSculpture_007.jpeg', 'HeadSculpture_008.jpeg', 'HeadSculpture_009.jpeg', 'HeadSculpture_010.jpeg', 'HeadSculpture_011.jpeg', 'HeadSculpture_012.jpeg', 'HeadSculpture_013.jpeg', 'HeadSculpture_014.jpeg', 'HeadSculpture_015.jpeg', 'HeadSculpture_016.jpeg', 'HeadSculpture_017.jpeg', 'HeadSculpture_018.jpeg', 'HeadSculpture_019.jpeg', 'HeadSculpture_020.jpeg', 'HeadSculpture_021.jpeg', 'HeadSculpture_022.jpeg', 'HeadSculpture_023.jpeg', 'HeadSculpture_024.jpeg', 'HeadSculpture_025.jpeg', 'HeadSculpture_026.jpeg', 'HeadSculpture_027.jpeg', 'HeadSculpture_028.jpeg']


In [10]:
cameras_new = create_spline_curve(cameras, 30)

In [18]:
# Camera origins
origins = cameras.origins().squeeze(-1)
origins_new = cameras_new.origins().squeeze(-1)

print(cameras.translation_vector.shape)
print(cameras.origins().shape)

torch.Size([28, 3, 1])
torch.Size([28, 3, 1])


### Colmap point 3D cloud

In [ ]:
points_3d_path = os.path.join(scene_dir, 'points3D.txt')
points_3d_sparse_cloud = parse_colmap_points_3d(points_3d_path)

## Create focal rays for cameras

In [15]:
# Focal axes for cameras
focal_axis_rays = FocalAxisRay(10., corner_rays=False)
focal_axis_rays.calc_ray_params(cameras)

lengths = sample_lengths(focal_axis_rays.origins.shape[0], 2, irregular=False, device='cpu', 
                         dtype=torch.float32)
points_3d_rays = sample_ray_points(
    focal_axis_rays.origins,
    focal_axis_rays.directions,
    lengths,
)

print(points_3d_rays.shape)

torch.Size([28, 2, 3])


In [13]:
# Resampling rays for new cameras
focal_axis_rays.calc_ray_params(cameras_new)

lengths = sample_lengths(focal_axis_rays.origins.shape[0], 2, irregular=False, device='cpu', 
                         dtype=torch.float32)
points_3d_rays_new = sample_ray_points(
    focal_axis_rays.origins,
    focal_axis_rays.directions,
    lengths,
)

## Ploting cameras and rays

In [14]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "scene"}]])

# Original cameras
fig.add_trace(go.Scatter3d(x=origins[:, 0], y=origins[:, 1], z=origins[:, 2]), row=1, col=1)

# Focal lines
for i in range(points_3d_rays.shape[0]):
    fig.add_trace(go.Scatter3d(x=points_3d_rays[i, 0:2, 0], y=points_3d_rays[i, 0:2, 1], z=points_3d_rays[i, 0:2, 2], mode='lines'))


fig.add_trace(go.Scatter3d(x=origins_new[:, 0], y=origins_new[:, 1], z=origins_new[:, 2]), row=1, col=2)

# Focal lines
for i in range(points_3d_rays_new.shape[0]):
    fig.add_trace(go.Scatter3d(x=points_3d_rays_new[i, 0:2, 0], y=points_3d_rays_new[i, 0:2, 1], z=points_3d_rays_new[i, 0:2, 2], mode='lines'), row=1, col=2)

fig.show()